<a href="https://colab.research.google.com/github/ajankowski/openvino/blob/master/00_full_openvino_pipeline_pic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://cdn-images-1.medium.com/fit/t/1600/480/1*_F330rP3SPZ-xhQKYQ1qhQ.png' height=100>

<img src='https://drive.google.com/uc?id=1w5dkGo71eFE_odBmUnvpC_BZ87u2rl2P'>

<img src='https://drive.google.com/uc?id=1SZGkdOqR0MCuquznMnkdMdDLLxgpIarX'>

<img src='https://drive.google.com/uc?id=123kbrrHJ4dHtMdpwBfkwSNDKZILtXNot'>

<img src='https://drive.google.com/uc?id=12tlvIVIrfrq_QyVpBUn0nIZxuPIg3qiZ'>

# Installing OpenVino and libraries import
<img src='https://miro.medium.com/max/3000/1*vdjibkLyilWAq-7CONW8Zg.png' height=100>



In [ ]:
# Install OpenVino

!wget https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2021
!touch /etc/apt/sources.list.d/intel-openvino-2021.list
!echo "deb https://apt.repos.intel.com/openvino/2021 all main" >> /etc/apt/sources.list.d/intel-openvino-2021.list
!pip install test-generator==0.1.1
!apt update
!apt install intel-openvino-dev-ubuntu18-2021.3

!pip install openvino

# Training the model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from PIL import Image

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import datasets

# MNIST data
<img src='https://miro.medium.com/max/584/1*2lSjt9YKJn9sxK7DSeGDyw.jpeg' height=200>

In [ ]:
(train_data, train_labels), (test_data, test_labels) = datasets.mnist.load_data()
data = train_data.astype(np.float32)
data = data /255
data = data.reshape(-1, 784)

classes = 10
labels = tf.keras.utils.to_categorical(train_labels, classes)

# model
<img src='https://cdn-images-1.medium.com/max/550/1*pO5X2c28F1ysJhwnmPsy3Q.gif' height=250>



In [ ]:
model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(784, )))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(data, labels, epochs=5)

# saving the model
converting a model saved in keras (.h5) to saved_model

In [ ]:
model.save('simple_model.h5') # keras API for saving model

model_ov = tf.keras.models.load_model('simple_model.h5')
tf.saved_model.save(model_ov,'saved_model')

<img src='https://cdn-images-1.medium.com/fit/t/1600/480/1*_F330rP3SPZ-xhQKYQ1qhQ.png' height=100>

<img src='https://drive.google.com/uc?id=1I3uejNH_hN5HmtDVYT2r0lnqJbPh0jmY'>

In [ ]:
mo_tf_path = '/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_tf.py'

model_dir = '/content/saved_model'
output_dir = '/content/'
input_shape = [1, 784]
input_shape_str = str(input_shape).replace(' ', '')

#Running Commands to run OPENVINO's Model Optimiser Converter 'mo_tf.py'
!python {mo_tf_path} --saved_model_dir {model_dir} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP16

In [ ]:
! ls 
# !cat /content/saved_model.xml

# sample digit

In [ ]:
i = 58

test_digit = test_data[i]
test_label = test_labels[i]

img = Image.fromarray(test_digit)
img.save('test_image.png')

plt.imshow(img)
plt.axis('off')
plt.title(f'label: {test_label}')
plt.show()

<img src='https://drive.google.com/uc?id=1M1wna8H8WyciiiOErvii3VmFE0Xk4t0O'>

**Due to running it in colab environment we have to run inference through bash script,  
 in a regular environment you would call python code directly.**



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from openvino.inference_engine import IECore

In [ ]:
# image preprocessing
def load_image(input_path):    
    capture = cv2.VideoCapture(input_path)       
    ret, image = capture.read()
    del capture    
    return image 

def prepare_image(image):
    resized = cv2.resize(image, (28, 28)) # resize image
    try:
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY) # change image to grayscale
    except:
        gray = resized
    reshaped = gray.reshape((1,) + ((gray.shape[0]*gray.shape[1]),)) # reshape from 28X28 to 1X784 (input_shape of our model)
    flipped = (255 - reshaped) /255 # flip color values and rescale from 0-255 to 0-1
    return reshaped

In [ ]:
# optimized model location
model_xml = '/content/saved_model.xml'
model_bin = '/content/saved_model.bin'

image_path = '/content/test_image.png'

# inference engine
ie = IECore()

net = ie.read_network(model=model_xml, weights=model_bin)
model_openvino = ie.load_network(network=net, num_requests=1, device_name='CPU')

# inputs and outputs
input_blob= next(iter(net.input_info)) 
output_blob = next(iter(net.outputs))

image = load_image(image_path)
prepared_image = prepare_image(image)

In [ ]:
# tf inference
model_tf = tf.keras.models.load_model('/content/simple_model.h5')
prediction_tf = model_tf.predict(prepared_image)

# openvino inference
prediction = model_openvino.infer(inputs={input_blob: prepared_image})

print(40*'-')
print(f'OpenVino prediction: {prediction}')
print(40*'-')
print(f'TensorFlow prediction: {prediction_tf}')

In [ ]:
%%time
prediction_tf = model.predict(prepared_image)

In [ ]:
%%time
prediction = model_openvino.infer(inputs={input_blob: prepared_image})

In [ ]:
%%timeit -r 3 -n 10
prediction_tf = model.predict(prepared_image)

In [ ]:
%%timeit -r 3 -n 10
prediction = model_openvino.infer(inputs={input_blob: prepared_image})